In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Загрузка данных
df_rating = pd.read_csv('C:/Users/Public/Documents/ratings.csv')
df_movies = pd.read_csv('C:/Users/Public/Documents/movies.csv')

# Объединение
df_rating = df_rating.merge(df_movies[['movieId', 'title']], on='movieId', how='left')

# Построение user-item матрицы
df_user_item = df_rating.pivot_table(index='userId', columns='title', values='rating')

# Матрица корреляций между фильмами
corr_matrix = df_user_item.corr(method='pearson', min_periods=100)

# Оценки пользователя 7
user_id = 7
user_ratings = df_user_item.loc[user_id].dropna()

# Список кандидатов
simCandidates = pd.Series(dtype='float64')

# Считаем "взвешенные" корреляции
for title, rating in user_ratings.items():
    sims = corr_matrix[title].dropna()
    sims = sims.map(lambda x: x * rating)
    simCandidates = pd.concat([simCandidates, sims])

# Суммируем повторы (дубликаты названий)
simCandidates = simCandidates.groupby(simCandidates.index).sum()

# Удаляем уже просмотренные фильмы
not_watched = simCandidates[~simCandidates.index.isin(user_ratings.index)]

# Рекомендуем топ-3
top3_recommendations = not_watched.sort_values(ascending=False).head(3)

print("🎬 Топ-3 рекомендации для пользователя 7:\n")
print(top3_recommendations)


🎬 Топ-3 рекомендации для пользователя 7:

Matrix, The (1999)                                                                30.707095
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    22.002139
Shawshank Redemption, The (1994)                                                  18.925514
dtype: float64
